In [1]:
# данные от Last.fm есть в открытом доступе, весят 543Мб - скачиваются какое-то время
!cd /home/inpefess/Downloads && wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz

--2017-11-11 13:50:07--  http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz
Resolving mtg.upf.edu (mtg.upf.edu)... 84.89.139.55
Connecting to mtg.upf.edu (mtg.upf.edu)|84.89.139.55|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569202935 (543M) [application/x-gzip]
Saving to: ‘lastfm-dataset-360K.tar.gz.1’

lastfm-dataset-360K 100%[===================>] 542,83M  1,83MB/s    in 6m 14s  

2017-11-11 13:56:21 (1,45 MB/s) - ‘lastfm-dataset-360K.tar.gz.1’ saved [569202935/569202935]



In [2]:
# распаковываем архив

!cd /home/inpefess/Downloads && tar -xvf /home/inpefess/Downloads/lastfm-dataset-360K.tar.gz

lastfm-dataset-360K/
lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
lastfm-dataset-360K/README.txt
lastfm-dataset-360K/mbox_sha1sum.py
lastfm-dataset-360K/usersha1-profile.tsv


In [3]:
!head /home/inpefess/Downloads/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv

00000c289a1829a808ac09c00daf10bc3c4e223b	3bd73256-3905-4f3a-97e2-8b341527f805	betty blowtorch	2137
00000c289a1829a808ac09c00daf10bc3c4e223b	f2fb0ff0-5679-42ec-a55c-15109ce6e320	die Ärzte	1099
00000c289a1829a808ac09c00daf10bc3c4e223b	b3ae82c2-e60b-4551-a76d-6620f1b456aa	melissa etheridge	897
00000c289a1829a808ac09c00daf10bc3c4e223b	3d6bbeb7-f90e-4d10-b440-e153c0d10b53	elvenking	717
00000c289a1829a808ac09c00daf10bc3c4e223b	bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8	juliette & the licks	706
00000c289a1829a808ac09c00daf10bc3c4e223b	8bfac288-ccc5-448d-9573-c33ea2aa5c30	red hot chili peppers	691
00000c289a1829a808ac09c00daf10bc3c4e223b	6531c8b1-76ea-4141-b270-eb1ac5b41375	magica	545
00000c289a1829a808ac09c00daf10bc3c4e223b	21f3573f-10cf-44b3-aeaa-26cccd8448b5	the black dahlia murder	507
00000c289a1829a808ac09c00daf10bc3c4e223b	c5db90c4-580d-4f33-b364-fbaa5a3a58b5	the murmurs	424
00000c289a1829a808ac09c00daf10bc3c4e223b	0639533a-0402-40ba-b6e0-18b067198b73	lunachicks	403


In [2]:
# загружаем данные
from os import path
import pandas as pd
import numpy as np

data_dir = "/home/inpefess/Downloads/lastfm-dataset-360K"

col_types = {
    "user": str,
    "artist-mbid": str,
    "artist-name": str,
    "total-plays": np.int32
}
data = pd.read_csv(
    path.join(data_dir, "usersha1-artmbid-artname-plays.tsv"),
    sep="\t",
    header=None,
    names=col_types.keys(),
    dtype=col_types
)
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [3]:
data.describe()

,total-plays
count,1.753566e+07
mean,2.151932e+02
std,6.144815e+02
min,0.000000e+00
25%,3.500000e+01
50%,9.400000e+01
75%,2.240000e+02
max,4.191570e+05


In [4]:
# заполняем пустые значения строкой None
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,37426
1,1099,1,152041
2,897,1,112367
3,717,1,38435
4,706,1,117444


In [5]:
# разобьём наблюдения на тестовую и обучающую выборки
test_indices = np.random.choice(data.index.values, replace=False, size=int(data.index.max() * 0.2))
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [6]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 9
нет в тестовой выборке, но есть в обучающей: 368
всего пользователей: 358868


In [7]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
test_data.drop(test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index, inplace=True)
train_data.drop(train_data[train_data["user_id"].isin(user_ids_to_exclude).values].index, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)

In [9]:
train_file_name = "train/lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, train_file_name),
    sep="\t",
    header=False,
    index=False
)

In [11]:
# создаём разреженную матрицу user*item
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

In [12]:
import time
from implicit.nearest_neighbours import CosineRecommender

model = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 6.915686368942261 секунд


In [13]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(data_dir, "recs", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 305.62073278427124 секнуд


In [14]:
# используем mrec в качестве метрики качества
!/home/inpefess/IdeaProjects/mrec/sbin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /home/inpefess/Downloads/lastfm-dataset-360K/lastfm.test.0 \
    --recsdir /home/inpefess/Downloads/lastfm-dataset-360K/recs

[2017-11-11 14:05:58,258] INFO: processing /home/inpefess/Downloads/lastfm-dataset-360K/lastfm.test.0...
None
mrr            0.1328 +/- 0.0000
prec@5         0.0490 +/- 0.0000
prec@10        0.0396 +/- 0.0000
prec@15        0.0264 +/- 0.0000
prec@20        0.0198 +/- 0.0000
